<h1> Assignment (Week 3) Capstone Project </h1><br><br>
<h1> Part I Retrieval and preparation of Toronto-Postal Code Data </h1>

<h3> After loading dependencies for this and later steps, <br>
The Beautiful-Soup Package was used to extract Data from Wikipedia. <br>
<bt>
Please keep in mind that there maybe issues with LXML-Parsing on some devices (it did on mine). <br>
This may require restarting your notebook after importing/sometimes installing/deinstalling the respective package <br>
<br>
Process: Irrelevant rows were removed from the data frame <br>
before editing the data frame (i.e. renaming empty Neighbourhood-Cells and grouping) <br> 
and finally printing the number of rows (i.e. relevant PostCodes)

In [ ]:
# Loading dependencies
from bs4 import BeautifulSoup # Used for wiki-scraping

import numpy as np # library to handle data in a vectorized manner
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

In [ ]:
# Loading required Toronto-Table from Wikipedia
wiki_url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
wiki = requests.get(wiki_url).text
html_soup=BeautifulSoup(wiki,'html.parser')
sov_tables = html_soup.find('table', class_="wikitable sortable")
df = pd.read_html(str(sov_tables))
df = df[0]
df.head()

In [ ]:
# Remove rows without assigned Borough
df=df[df.Borough != 'Not assigned']


In [ ]:
# Replace 'Not Assigned' Neighbourhoods with corresponding 'Borough'-Entry
df.Neighbourhood=df.Neighbourhood.replace('Not assigned',df.Borough)

In [ ]:
# Create and check new dataframe by grouping based on PostCode and join Neighbourhood entrys
df_Toronto=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_Toronto.head(5)

<h2> Solution to Part I; Summary of changes to data frame </h2>

In [ ]:
# Check Shape
df_Toronto.shape

In [ ]:
print('The grouped Data frame contains {} Rows (i.e. Postalcodes) as opposed to the non-grouped data frame, which contained {} rows'.format(df_Toronto.shape[0],df.shape[0]))

<h1> Part II Matching Postal-Codes to Location Data </h1>

In [ ]:
# Load CSV Data (as geocoder happened to be to unreliable)
import wget

url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
wget.download(url, 'Geospatial_Coordinates.csv')

print ('File downloaded and saved!')

In [ ]:
# Edit Coordinate-Table to allow mergin
Geospatial_Coordinates = pd.read_csv('Geospatial_Coordinates.csv')
Geospatial_Coordinates.columns = ['Postcode','Latitude','Longitude']

In [ ]:
# Merge Coordinate Data with Neigbourhood/Toronto-Dataframe
df_ToCo = pd.merge(df_Toronto, Geospatial_Coordinates, on='Postcode')

<h2> Solution for Part II: Matched Postcodes to Location Data

In [ ]:
# Check Result
df_ToCo.head(15)

<h1> Part III: Visualizing

<h3> In this part the Foursquare API is used to produce a Map based on offered services. </h3>

<h4>
<br> Note, that I decided against limiting the scope to some particular Boroughs, as I did not see any particular reason to do so </h4>
<br> This led to some later changes, as I dealt with neighbourhoods without surrounding venues
<br> These were later grouped to an additional cluster</h4>



<h3> Start preparing Foursquare data retrieval and consequent clustering 

In [ ]:
CLIENT_ID = 'TKZUDAD3BGL5SQUTQJ31AF1RO5FNLE1XB0QP23EKN3H0WLIR' # your Foursquare ID
CLIENT_SECRET = 'GGHSHKUOEXMK2E4HHLVZW0THPQODRZKS3LXBU3DMRIJ5HDVP' # your Foursquare Secret
VERSION = '20200513' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
radius = 500 # Define radius of interest: AKA how lazy do you want to be to reach your favorite pizza shop (or alike)?
LIMIT = 100 # How many venues do you want to check?

<h3> Define function to retrieve most common venues for each Neighbourhood </h3>

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# type your answer here

Toronto_venues = getNearbyVenues(names=df_ToCo['Neighbourhood'],
                                   latitudes=df_ToCo['Latitude'],
                                   longitudes=df_ToCo['Longitude']
                                  )



In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

<h4> Group the Dummy-Data to find out the relative frequency of a venue category per Neighbourhood,
<br> define a function to return the n-topmost venues
<br> use this function to create a data frame, including the 10-topmost venues for each neighbourhood

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Deli / Bodega,Thai Restaurant,Gym,Sushi Restaurant,Bakery
1,Agincourt,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Department Store,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Sculpture Garden,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fried Chicken Joint,Pharmacy,Beer Store,Fast Food Restaurant,Sandwich Place,Pizza Place,Liquor Store,Concert Hall,Construction & Landscaping
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Sandwich Place,Athletics & Sports,Pharmacy,Coffee Shop,Gym,Pub,Drugstore,Donut Shop


In [155]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

In [156]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_ToCo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

<h3> As I encountered some problems later in the process; I figured I had to check for missing values
<br> Apparently there are neighbourhoods with no venue in 500m proximity
<br> I then decided to add an additional cluster for all the neighbourhoods where no venue was listed within 500m


In [157]:
# Show NA - Clusters
Toronto_merged[Toronto_merged['Cluster Labels'].isna()]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
# Replace NA-Labels with "5" as a manually added (yet meaningful) label
Toronto_merged['Cluster Labels'].fillna(5,inplace = True)

In [159]:
# create a map, centered around the mean (i.e. Median) toronto Latitude and Longitude Values

Toronto_mdLat = df_ToCo['Latitude'].median()
Toronto_mdLong = df_ToCo['Longitude'].median()

map_clusters = folium.Map(location=[Toronto_mdLat, Toronto_mdLong], zoom_start=11)

# set color scheme for the clusters / note I changed it compared to the Manhattan - Task; as I added a "missing value category i.e. No Venues in 500m radius"
x = np.arange(Toronto_merged['Cluster Labels'].nunique())
ys = [i + x + (i*x)**2 for i in range(Toronto_merged['Cluster Labels'].nunique())]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [149]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Intersection,Electronics Store,Mexican Restaurant,Breakfast Spot,Bank,Rental Car Location,Medical Center,Yoga Studio,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Bakery,Bank,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Diner,Dessert Shop
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,4.0,Discount Store,Train Station,Convenience Store,Hobby Shop,Department Store,Coffee Shop,Eastern European Restaurant,Drugstore,Donut Shop,Electronics Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,4.0,Bakery,Bus Line,Bus Station,Soccer Field,Park,Intersection,Ice Cream Shop,Yoga Studio,Discount Store,Diner
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,4.0,General Entertainment,College Stadium,Skating Rink,Café,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
10,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,4.0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Light Rail Station,Brewery,Eastern European Restaurant,Drugstore,Electronics Store,Donut Shop
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,4.0,Middle Eastern Restaurant,Breakfast Spot,Bakery,Sandwich Place,Auto Garage,Shopping Mall,Drugstore,Donut Shop,Eastern European Restaurant,Department Store
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,4.0,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Department Store,Donut Shop
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,4.0,Pizza Place,Coffee Shop,Fried Chicken Joint,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Rental Car Location,Gas Station,Convenience Store,Pharmacy
